<img src="images/gibs.png" alt="NASA GIBS Logo"></img>

Image from [NASA Global Imagery Browse Services (GIBS) GitHub](https://github.com/nasa-gibs)

# NASA Earthdata GIBS Explorer

Global Imagery Browse Services (GIBS) provides quick access to over 1,000 satellite imagery products, covering every part of the world. Most imagery is updated daily—available within a few hours after satellite observation, and some products span almost 30 years.

Below demos how to use OWSLib, Geoviews, and Panel effectively to create our own GIBS explorer.

## Prerequisites

The following packages are good to know, but not required.

| Concepts | Importance | Notes |
| --| --| --- |
| [Intro to GeoViews](https://geoviews.org/) | Helpful | Geographic visualizations |
| [Intro to Panel](https://panel.holoviz.org/) | Helpful | Dashboard creations |
| [Intro to OWSLib](https://owslib.readthedocs.io/en/latest/usage.html) | Helpful | WMS URLs |

- **Time to learn**: 15 minutes

---

## Imports

Let’s first import a few packages.

GeoViews is a Python library that facilitates the integration of WMS and other geospatial data sources with your own datasets. It provides a high-level interface for working with geographic data and simplifies the process of creating interactive visualizations.

Pandas is a powerful Python library for data manipulation and analysis. It offers versatile data structures, such as Series and DataFrame, for working with structured data. However, here, we will only be using it to generate date time ranges.

Panel is a Python library that offers a set of flexible and powerful tools for creating interactive dashboards and apps. It allows you to build custom user interfaces with interactive controls, widgets, and layout components, enabling rich interactivity for your visualizations and data analysis workflows.

OWSLib is a Python library designed for client-side programming using the interface standards of the Open Geospatial Consortium (OGC) web services and their associated content models. Specifically, in this scenario, OWSLib will be utilized solely for the purpose of constructing URLs for WMS.

The next line, `gv.extension("bokeh")`, enables the Bokeh (interactive) plotting backend for GeoViews. GeoViews supports multiple plotting backends, such as Bokeh and Matplotlib, which allow you to choose the one that best suits your needs.

Finally, `pn.extension()` initializes the panel library and sets up the necessary environment for creating interactive panels and dashboards. You may specify configurations like `sizing_mode="stretch_width"` within `pn.extension()`.

In [ ]:
import panel as pn
import pandas as pd
import geoviews as gv
from owslib.wms import WebMapService

gv.extension("bokeh")
pn.extension(sizing_mode="stretch_width")

# Accessing GIBS

Accessing NASA's GIBS (Global Imagery Browse Services) is well-documented, and you can find the documentation [here](https://nasa-gibs.github.io/gibs-api-docs/access-basics/).

To access GIBS through the WMS (Web Map Service) endpoints, you can follow these steps:

1. Find the WMS service endpoints by referring to the [service endpoints section](https://nasa-gibs.github.io/gibs-api-docs/access-basics/#service-endpoints_1) of the documentation. Look for the row that corresponds to the EPSG:3857 projection, as GeoViews currently supports that projection for tile services.

2. Once you have identified the WMS service endpoint, copy one of the versions' [GetCapabilities URLs](https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi?SERVICE=WMS&REQUEST=GetCapabilities&VERSION=1.3.0). This URL provides information about the available layers and operations supported by the WMS service.

3. Pass the GetCapabilities URL to the `WebMapService` class, which is a part of the `OWSLib` library. This class allows you to interact with the WMS service and retrieve the desired data.

By following these steps, you will be able to access and work with the NASA GIBS data using the WMS service endpoints.

In [ ]:
base_resource_url = "https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi?SERVICE=WMS&REQUEST=GetCapabilities&VERSION=1.3.0"

wms = WebMapService(base_resource_url)

If we examine the contents, we can see that there are over a 1,000 layers (products) available!

In [ ]:
wms_contents = pd.Series(wms.contents)
print(len(wms_contents))
wms_contents.index

With a myriad of captivating options within your reach, why not embark on a journey of exploration and create your own interactive explorer?

Now, you might be wondering, since there already exists an online explorer called [WorldView](https://worldview.earthdata.nasa.gov/), why bother reinventing the wheel? Well, here's the catch: by building your own explorer, you have the freedom to incorporate your own datasets into the mix!

Not only does this provide a unique opportunity to personalize your exploration experience, but it's also a fantastic way to explore all the exciting options available while showcasing the incredible power of Python packages working in harmony!

In [ ]:
BASE_URL = "https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi?SERVICE=WMS"
XMIN = -20037507.539400
YMIN = 1638517.444800
XMAX = 20037260.918700
YMAX = 7714669.39460


class NasaEarthDataWmsExplorer:
    def __init__(self):
        self.wms = WebMapService(BASE_URL)
        layers = list(self.wms.contents)

        # create widgets
        self.layer_input = pn.widgets.Select(
            name="Layer",
            options=layers,
        )
        self.time_slider = pn.widgets.DiscreteSlider(name="Time", margin=(5, 16))
        self.coastline_feature = gv.feature.coastline().opts(
            global_extent=True, responsive=True
        )
        self.static_text = pn.widgets.StaticText()
        self.image_pane = pn.pane.Image()
        self.holoviews_pane = pn.pane.HoloViews(
            object=self.coastline_feature, min_height=500, sizing_mode="stretch_both"
        )

        # add interactivity
        self.layer_input.param.watch(self.update_time, "value")
        self.time_slider.param.watch(self.update_web_map, "value_throttled")

    def update_time(self, event):
        layer = event.new
        time_positions = self.wms.contents[layer].timepositions
        if not time_positions:
            # use N/A instead of None to circumvent Panel from crashing
            # when going from time-dependent layer to time-independent layer
            options = ["N/A"]
            value = "N/A"
        else:
            ini, end, step = time_positions[0].split("/")
            options = (
                pd.date_range(ini, end, freq=pd.Timedelta(step))
                .strftime("%Y-%m-%dT%H:%M:%SZ")
                .tolist()
            )
            value = options[0]
        self.time_slider.param.set_param(options=options, value=value)
        self.update_web_map()

    def update_web_map(self, event=None):
        try:
            self.holoviews_pane.loading = True
            layer = self.layer_input.value
            time = self.time_slider.value
            if time == "N/A":
                time = None

            url = wms.getmap(
                layers=[layer],
                srs="EPSG:3857",
                bbox=(XMIN, YMIN, XMAX, YMAX),
                size=(256, 256),
                format="image/png",
                transparent=True,
                time=time,
            ).geturl()
            url_template = (
                url.replace(str(XMIN), "{XMIN}")
                .replace(str(YMIN), "{YMIN}")
                .replace(str(XMAX), "{XMAX}")
                .replace(str(YMAX), "{YMAX}")
            )

            layer_meta = self.wms[layer]
            self.image_pane.object = layer_meta.styles.get("default", {}).get("legend")
            self.static_text.value = layer_meta.abstract
            layer_imagery = gv.WMTS(url_template).opts(title=layer_meta.title)

            self.holoviews_pane.object = self.coastline_feature * layer_imagery
        finally:
            self.holoviews_pane.loading = False

    def view(self):
        widget_box = pn.WidgetBox(
            self.layer_input,
            self.time_slider,
            self.image_pane,
            self.static_text,
            pn.Spacer(sizing_mode="stretch_height"),
            sizing_mode="stretch_both",
        )
        return pn.Row(
            widget_box,
            self.holoviews_pane,
        )


nasa_earth_data_wms_explorer = NasaEarthDataWmsExplorer()
nasa_earth_data_wms_explorer.view()

No such comm: 90be3a54996442ad9c86f9abc1f60f70


The provided code allows users to interactively explore various layers of NASA Earth Data imagery.

The `NasaEarthDataWmsExplorer` uses `WebMapService` from `OWSLib` ibrary to connect to the NASA Earth Data WMS service. The available layers are retrieved and displayed in a select widget.

The explorer provides interactivity through `panel` widgets such as the layer selection dropdown and the time slider.

Selecting a layer updates the available time positions for that layer, while changing the time position updates the displayed imagery accordingly. Metadata from the layer is also extracted and displayed below the widgets.

The imagery is displayed using the GeoViews library, combined with a coastline feature.


## Summary

While the standalone nature of this custom-built explorer may not rival the capabilities of the existing WorldView explorer, its true potential lies in the ability to incorporate your own data. By integrating your unique datasets into this explorer, you can unlock a world of fascinating possibilities and create a truly captivating exploration experience.

It is this integration of personal data that sets this explorer apart and makes it incredibly compelling and engaging!